
# Appliances Energy Prediction — Análises em Python (Parte 3)

Este notebook cobre as tarefas 26 a 35 usando o dataset "Appliances energy prediction" da UCI:
https://archive.ics.uci.edu/dataset/374/appliances+energy+prediction

O notebook inclui download automático do arquivo, carregamento, análises exploratórias, PCA, modelagem e avaliação.


In [ ]:

# Tentativa de download do dataset da UCI se não existir localmente.
import os, urllib.request, zipfile, io, pandas as pd

DATA_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv"
FNAME = "energydata_complete.csv"

if not os.path.exists(FNAME):
    try:
        print("Baixando dataset...")
        urllib.request.urlretrieve(DATA_URL, FNAME)
        print("Download concluído:", FNAME)
    except Exception as e:
        print("Falha ao baixar automaticamente. Faça o download manualmente:", e)
else:
    print("Arquivo encontrado:", FNAME)


## Imports e configurações

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
pd.set_option('display.max_columns', None)


## 26) Carregamento e inspeção inicial

In [ ]:

df = pd.read_csv('energydata_complete.csv')
print('Formato:', df.shape)
print('\n.info()\n')
print(df.info())
print('\n.describe()\n')
df.describe().T



## 27) Distribuição do consumo (`Appliances`)
- Histogramas e série temporal.


In [ ]:

# Histograma
plt.figure(figsize=(6,4))
plt.hist(df['Appliances'], bins=50)
plt.title('Histograma de Appliances')
plt.xlabel('Appliances (Wh)')
plt.ylabel('Frequência')
plt.tight_layout()
plt.show()

# Série temporal (amostragem dos primeiros 7 dias para visualização)
df['date'] = pd.to_datetime(df['date'])
plt.figure(figsize=(10,4))
sample_ts = df.set_index('date')['Appliances'].iloc[:24*7]  # primeiros 7 dias
plt.plot(sample_ts.index, sample_ts.values)
plt.title('Série temporal - Appliances (primeira semana)')
plt.xlabel('Data')
plt.ylabel('Appliances (Wh)')
plt.tight_layout()
plt.show()



## 28) Correlações entre `Appliances` e variáveis ambientais (temperatura, umidade...)


In [ ]:

# Seleciona variáveis ambientais relevantes do dataset
env_vars = ['T1','RH_1','T2','RH_2','T3','RH_3','T_out','RH_out','Windspeed','Press_mm_hg','Tdewpoint']
# Nem todos podem existir; filtramos os que estão presentes
env_vars = [v for v in env_vars if v in df.columns]

corrs = df[['Appliances'] + env_vars].corr().loc['Appliances', env_vars].sort_values(ascending=False)
corrs



## 29) Normalização (Min-Max Scaling) das variáveis numéricas
- Aplicar Min-Max e armazenar em `df_scaled` para uso posterior.


In [ ]:

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
num_cols.remove('Appliances') if 'Appliances' in num_cols else None

scaler = MinMaxScaler()
df_scaled = df.copy()
df_scaled[num_cols] = scaler.fit_transform(df_scaled[num_cols])

# também escalamos Appliances para consistência se quisermos (opcional)
df_scaled['Appliances_scaled'] = MinMaxScaler().fit_transform(df[['Appliances']])

print('Exemplo de dados normalizados:')
df_scaled[num_cols + ['Appliances_scaled']].head()



## 30) PCA — reduzir para 2 componentes principais
- Aplicar PCA nas variáveis numéricas (excluindo `date`) e plotar os pontos.


In [ ]:

from sklearn.preprocessing import StandardScaler
pca_cols = df.select_dtypes(include=[np.number]).columns.tolist()
pca_cols = [c for c in pca_cols if c not in ['Appliances_scaled','Appliances']]

X = df[pca_cols].dropna().astype(float)
X_scaled = StandardScaler().fit_transform(X)

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

print('Variância explicada por componente:', pca.explained_variance_ratio_)

plt.figure(figsize=(8,6))
plt.scatter(X_pca[:,0], X_pca[:,1], s=10, alpha=0.5)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA (2 componentes) — Appliances dataset')
plt.grid(alpha=0.3)
plt.show()



## 31) Regressão Linear Múltipla — prever `Appliances` a partir das variáveis ambientais


In [ ]:

# Preparar features: usamos variáveis numéricas exceto date and Appliances
features = [c for c in df.select_dtypes(include=[np.number]).columns if c != 'Appliances']
X = df[features].astype(float).fillna(method='ffill').fillna(method='bfill')
y = df['Appliances'].astype(float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

rmse_lr = mean_squared_error(y_test, y_pred, squared=False)
r2_lr = r2_score(y_test, y_pred)
rmse_lr, r2_lr



## 32) Random Forest Regressor — comparar RMSE com regressão linear


In [ ]:

rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
r2_rf = r2_score(y_test, y_pred_rf)
rmse_rf, r2_rf



## 33) K-Means clustering (3 a 5 clusters) — interpretar perfis de consumo


In [ ]:

from sklearn.metrics import silhouette_score

# Usar apenas features numéricas reduzidas por StandardScaler
kmeans_X = StandardScaler().fit_transform(df.select_dtypes(include=[np.number]).fillna(0))

results = {}
for k in range(3,6):
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(kmeans_X)
    sil = silhouette_score(kmeans_X, labels)
    results[k] = {'kmeans': km, 'labels': labels, 'silhouette': sil}

results_summ = {k: {'silhouette': results[k]['silhouette'], 'sizes': np.bincount(results[k]['labels'])} for k in results}
results_summ



## 34) Classificação binária — alto vs baixo consumo (mediana)
- Treinar Logistic Regression e Random Forest Classifier.


In [ ]:

# Criar target binário
median_val = df['Appliances'].median()
df_class = df.copy()
df_class['high_consumption'] = (df_class['Appliances'] > median_val).astype(int)

# Usar mesmas features que antes (numéricas)
features = [c for c in df_class.select_dtypes(include=[np.number]).columns if c not in ['Appliances','high_consumption']]
Xc = df_class[features].fillna(0)
yc = df_class['high_consumption']

Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size=0.2, random_state=42, stratify=yc)

# Logistic Regression
logr = LogisticRegression(max_iter=1000)
logr.fit(Xc_train, yc_train)
yc_pred_log = logr.predict(Xc_test)

# Random Forest Classifier
rfc = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rfc.fit(Xc_train, yc_train)
yc_pred_rf = rfc.predict(Xc_test)

print('Logistic trained, RandomForest trained')



## 35) Avaliação de classificação — matriz de confusão e métricas


In [ ]:

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

print('=== Logistic Regression ===')
print('Accuracy:', accuracy_score(yc_test, yc_pred_log))
print(classification_report(yc_test, yc_pred_log))
print('Confusion matrix:\n', confusion_matrix(yc_test, yc_pred_log))

print('\n=== Random Forest ===')
print('Accuracy:', accuracy_score(yc_test, yc_pred_rf))
print(classification_report(yc_test, yc_pred_rf))
print('Confusion matrix:\n', confusion_matrix(yc_test, yc_pred_rf))



### Observações rápidas
- O dataset contém medições em Wh e variáveis ambientais internas/externas.  
- Em modelos de regressão, Random Forest costuma superar Regressão Linear quando há relações não-lineares.  
- Em classificação binária, analise qual classe apresenta mais erros através da matriz de confusão (ex.: `False Negative` indica alto consumo classificado como baixo).
